In [1]:
from underthesea import dependency_parse

In [2]:
subject_blacklist = {"tôi", "mình", "em", "anh", "chị", "bạn"}

In [9]:
def extract_attributes(text, subject_blacklist=None, debug=False):
    """
    Trích xuất các cặp (chủ ngữ -> vị ngữ mô tả) từ kết quả dependency parse,
    loại bỏ các chủ ngữ nằm trong blacklist.

    Args:
        dep_parse_output (list): Danh sách các tuple từ dependency_parse().
                                 Ví dụ: [('Sản phẩm', 3, 'nsubj'), ...]
        subject_blacklist (set, optional): Tập hợp các chủ ngữ (dạng chữ thường)
                                           cần loại bỏ. Mặc định là {'tôi', 'mình', ...}.

    Returns:
        dict: Dictionary chứa các cặp (chủ ngữ -> vị ngữ mô tả).
              Ví dụ: {'Sản phẩm': 'tốt nhất', 'pin': 'dùng lâu'}
    """

    parse_output = dependency_parse(text)

    # In ra để dễ theo dõi (tùy chọn)
    if debug:
        print("Kết quả Dependency Parse:")
        for item in parse_output:
            print(item)
        print("-" * 20)

    # Xử lý kết quả
    results = {}

    # Tạo một danh sách các token với chỉ số 0-based để dễ truy cập
    # Thêm trường 'id' (chỉ số gốc 0-based) và chuyển 'head' thành 0-based
    indexed_parse = []
    for i, (word, head_idx, rel) in enumerate(parse_output):
        indexed_parse.append(
            {
                "id": i,
                "word": word,
                "head": head_idx
                - 1,  # Chuyển sang 0-based index (-1 nếu head là 0/root)
                "rel": rel,
            }
        )

    # Tìm các chủ ngữ (nsubj)
    for i, token in enumerate(indexed_parse):
        if token["rel"] == "nsubj":
            subject_word = token["word"]
            head_index = token["head"]  # Chỉ số 0-based của head

            subject_word_lower = (
                subject_word.lower()
            )  # Chuyển sang chữ thường để so sánh

            # === BƯỚC LỌC: Bỏ qua nếu chủ ngữ nằm trong blacklist ===
            if subject_word_lower in subject_blacklist:
                if debug:
                    print(f"-> Bỏ qua chủ ngữ '{subject_word}' vì nằm trong blacklist.")
                
                continue  # Bỏ qua vòng lặp này và xử lý token tiếp theo

            # Đảm bảo head_index hợp lệ
            if 0 <= head_index < len(indexed_parse):
                head_token = indexed_parse[head_index]
                predicate_parts = [head_token]  # Bắt đầu cụm vị ngữ bằng head

                # Tìm các thành phần phụ thuộc trực tiếp vào head_token này
                # để mở rộng cụm vị ngữ (ví dụ: acomp, advmod, obj...)
                for potential_dependent in indexed_parse:
                    # Nếu token này phụ thuộc vào head_token hiện tại
                    if potential_dependent["head"] == head_token["id"]:
                        # Chỉ lấy các quan hệ bổ nghĩa cho vị ngữ
                        # (Bạn có thể thêm các loại quan hệ khác nếu cần)
                        if potential_dependent["rel"] in [
                            "acomp",
                            "advmod",
                            "obj",
                            "xcomp",
                        ]:
                            predicate_parts.append(potential_dependent)

                # Sắp xếp các phần của vị ngữ theo thứ tự xuất hiện trong câu gốc
                predicate_parts.sort(key=lambda x: x["id"])

                # Ghép các từ thành chuỗi vị ngữ
                predicate_string = " ".join([p["word"] for p in predicate_parts])

                # Lưu kết quả (chuyển sang chữ thường cho nhất quán)
                results[subject_word.lower()] = predicate_string.lower()

    # In kết quả cuối cùng
    if debug:
        print("Kết quả trích xuất:")
        for subject, predicate in results.items():
            print(f"{subject} -> {predicate}")
        print("-" * 20)

    return results

In [16]:
sentence = "Sản phẩm tốt nhất mà tôi từng mua, pin dùng lâu nhưng giá hơi cao, kết nối mạng kém, máy nóng, nhân viên tư vấn nhiệt tình"

extract_attributes(sentence, subject_blacklist, True)

Kết quả Dependency Parse:
('Sản phẩm', 2, 'nsubj')
('tốt', 0, 'root')
('nhất', 2, 'advmod')
('mà', 7, 'mark')
('tôi', 7, 'nsubj')
('từng', 7, 'advmod')
('mua', 2, 'ccomp')
(',', 10, 'punct')
('pin', 10, 'nsubj')
('dùng', 7, 'conj')
('lâu', 10, 'acomp')
('nhưng', 15, 'mark')
('giá', 15, 'nsubj')
('hơi', 15, 'advmod')
('cao', 10, 'conj')
(',', 18, 'punct')
('kết nối', 18, 'nummod')
('mạng', 15, 'conj')
('kém', 18, 'nmod')
(',', 21, 'punct')
('máy', 18, 'conj')
('nóng', 21, 'amod')
(',', 24, 'punct')
('nhân viên', 18, 'conj')
('tư vấn', 24, 'acl:subj')
('nhiệt tình', 25, 'compound:svc')
--------------------
-> Bỏ qua chủ ngữ 'tôi' vì nằm trong blacklist.
Kết quả trích xuất:
sản phẩm -> tốt nhất
pin -> dùng lâu
giá -> hơi cao
--------------------


{'sản phẩm': 'tốt nhất', 'pin': 'dùng lâu', 'giá': 'hơi cao'}

In [49]:
extract_attributes("Cũng tạm. Hàng trưng bày có bị xước màn hình rồi nhưng mọi thứ dùng ổn. Pin hơi đuối. Tóm lại giá rẻ hơn mọi chỗ, máy zin all, chế độ bảo hành đổi trả kém hơn chỗ khác. OK", subject_blacklist, True)

Kết quả Dependency Parse:
('Cũng', 2, 'advmod')
('tạm', 0, 'root')
('.', 2, 'punct')
('Hàng', 8, 'nsubj')
('trưng bày', 4, 'nmod')
('có', 8, 'advmod')
('bị', 8, 'aux')
('xước', 2, 'parataxis')
('màn hình', 8, 'obj')
('rồi', 8, 'advmod')
('nhưng', 14, 'mark')
('mọi', 13, 'det')
('thứ', 14, 'nsubj')
('dùng', 2, 'conj')
('ổn', 14, 'obj')
('.', 14, 'punct')
('Pin', 18, 'nummod')
('hơi đuối', 14, 'obj')
('.', 20, 'punct')
('Tóm lại', 14, 'conj')
('giá', 20, 'nmod')
('rẻ', 21, 'acl:subj')
('hơn', 22, 'advmod:adj')
('mọi', 25, 'det')
('chỗ', 22, 'obl:adj')
(',', 27, 'punct')
('máy', 25, 'conj')
('zin all', 27, 'compound')
(',', 30, 'punct')
('chế độ', 14, 'conj')
('bảo hành', 30, 'compound')
('đổi', 31, 'acl:subj')
('trả', 32, 'xcomp')
('kém', 33, 'acomp')
('hơn', 36, 'advmod:adj')
('chỗ', 34, 'obl:adj')
('khác', 36, 'amod')
('.', 2, 'punct')
('OK', 2, 'punct')
--------------------
Kết quả trích xuất:
hàng -> có xước màn hình rồi
thứ -> dùng ổn hơi đuối
--------------------


{'hàng': 'có xước màn hình rồi', 'thứ': 'dùng ổn hơi đuối'}